In [1]:
import numpy as np
import numpy.random
import numpy.matlib
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd # Pandas library
import numpy as np # Numpy library
import matplotlib.pyplot as plt # Matplotlib library
import numpy.linalg as la # Linear algebra functions
import math # Math library
import random # Random library
import seaborn as sns # Seaborn library
crime_data = pd.read_csv('../SanFranciscoCrimeDataset/crime.csv',
                          header=1,
                          skipinitialspace=False,
                          names=['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
                                'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'])
crime_data = crime_data[ ~crime_data['PdDistrict'].isna() ]

In [2]:


#Time
crime_data_timeMin = [((int(i.split(':')[0])*60) + int(i.split(':')[1])) for i in crime_data['Time']]
crime_data['Time in Min'] = crime_data_timeMin
print("Done Time")

#Date
crime_data_date = [i.split()[0] for i in crime_data['Date']]
crime_data_dateDays = [ ((int(i.split('/')[0])-1)*30.4167) + (int(i.split('/')[1])) for i in crime_data_date]
crime_data['Date in Days'] = crime_data_dateDays
print("Done Date")

#PD District
District_Parse = list(set([i[0] for i in crime_data[['PdDistrict']].values]))
crime_data_District = [np.array([0 if District_Parse.index(d)!=i else\
 1 for (i,p) in enumerate(District_Parse)]) for d in\
crime_data[['PdDistrict']].values]

crime_data['One Hot Encoding PdDist'] = crime_data_District
print("Done PD District")

#Day of week
Day_Parse = list(set([i[0] for i in crime_data[['DayOfWeek']].values]))
crime_data_Day = [np.array([0 if Day_Parse.index(d)!=i else 1 for\
(i,p) in enumerate(Day_Parse)]) for d in crime_data[['DayOfWeek']]\
.values]

crime_data['One Hot Encoding DayOfWeek'] = crime_data_Day

print("Done Day of Week")

#Category
Category_Parse = list(set([i[0] for i in crime_data[['Category']].values]))
crime_data_Category = [np.array([0 if Category_Parse.index(d)!=i\
else 1 for (i,p) in enumerate(Category_Parse)]) for d in\
crime_data[['Category']].values]

crime_data['One Hot Encoding Category'] = crime_data_Category

print("Done")

Done Time
Done Date
Done PD District
Done Day of Week
Done


In [3]:
def split_sets(dataframe, train_p):
    """
    Returns 2 lists of the form needed to use the KNN class using the San Francisco Crime dataset
    -- The function uses x and y coordinate data for graphing and the type of crime as our labels.
    **The dataframe passed can be subset to go by location to see how KNN works on each "area"
    """
    assert(train_p < 1) # Make sure that p is a percentage
    assert(train_p > 0) # Make sure that p is non-negative
    
    # Split dataframe into training and test/prediction sets
    train_count = int(dataframe.shape[0] * train_p)
    predict_count = int(dataframe.shape[0] * (1 - train_p))
    train_count_Y = int(dataframe.shape[0] * train_p)
    predict_count_Y = int(dataframe.shape[0] * (1 - train_p))
    
    
    tmp_list = dataframe.values[:]
    np.random.shuffle(tmp_list) # Shuffle the rows
    train_list = tmp_list[0:train_count] # Take the first train_p percentage for the training data
    predict_list = tmp_list[train_count:] # The rest go to predict data
    

    rtn_train = [[i[9], i[10], i[13], i[14], *i[15], *i[16]] for i in train_list]
    rtn_predict = [[i[9], i[10], i[13], i[14], *i[15], *i[16]]for i in predict_list]
    
    rtn_train_Y = [i[17] for i in train_list]
    rtn_predict_Y = [ i[17] for i in predict_list]
    
    return rtn_train, rtn_predict, rtn_train_Y, rtn_predict_Y


train_NN1, predict_NN1, train_NN1_Y, predict_NN1_Y = split_sets(crime_data, .3)

train_NN = np.array(train_NN1)
train_NN_Y = np.array(train_NN1_Y)

print(train_NN)
print(train_NN_Y)

[[ -1.22398580e+02   3.77920266e+01   1.23000000e+03 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [ -1.22406412e+02   3.77802525e+01   6.00000000e+01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.22407878e+02   3.77859680e+01   8.50000000e+02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [ -1.22424451e+02   3.77671836e+01   1.95000000e+02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.22468171e+02   3.77437320e+01   7.20000000e+02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.22431046e+02   3.77830296e+01   1.32000000e+03 ...,   0.00000000e+00
    1.00000000e+00   0.00000000e+00]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [4]:
from nn import NN
import importlib # needed to reload a module
#importlib.reload(nn) # reload to get the new defs

In [ ]:
parse_crimes = list(set([i[0] for i in crime_data[['Category']].values]))

test = NN(21, 2,2,  .1, True, len(parse_crimes))
test.reset()
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 1)
print("Done")

Current loss: 2692.35254157
Done


In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 1000)

Current loss: 869.445625926


In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 1000)

In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 10000)

In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 10000)

In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 10000)

In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 100000)

In [ ]:
test.train_rounds(train_NN[:1000], train_NN_Y[:1000], 500000)